In [15]:
!pip install alpaca-trade-api

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import alpaca_trade_api as tradeapi
import yfinance as yf

API_KEY = 'PKW9E7ALGQBKQS1OT65O'
API_SECRET = 'q13WxcoNeMjjJOnq0IwdyPxw2qe4WNTAI5ySICpn'
BASE_URL = 'https://paper-api.alpaca.markets'

api = tradeapi.REST(API_KEY, API_SECRET, BASE_URL, api_version='v2')

def time_series_momentum_strategy(data):
    data['returns'] = data['Close'].pct_change()
    data['rolling_returns'] = data['returns'].rolling(window=20).sum()
    data['signal'] = data['rolling_returns'].apply(lambda x: 1 if x > 0 else -1)
    return data

def execute_trades(symbol, signal):
    account = api.get_account()
    cash = float(account.cash)

    position = api.get_position(symbol)
    if position:
        current_quantity = int(position.qty)
    else:
        current_quantity = 0

    if signal == 1 and current_quantity <= 0:
        quantity = int(cash / data['Close'].iloc[-1])
        api.submit_order(
            symbol=symbol,
            qty=quantity,
            side='buy',
            type='market',
            time_in_force='gtc'
        )
        st.write(f"Bought {quantity} shares of {symbol}")
    elif signal == -1 and current_quantity > 0:
        api.submit_order(
            symbol=symbol,
            qty=abs(current_quantity),
            side='sell',
            type='market',
            time_in_force='gtc'
        )
        st.write(f"Sold {abs(current_quantity)} shares of {symbol}")

st.title("Trading Chatbot")

symbol = st.text_input("Enter the symbol")

if st.button("Execute Trades"):
    start_date = '2022-01-01'
    end_date = '2022-12-31'

    data = yf.download(symbol, start=start_date, end=end_date)
    data = time_series_momentum_strategy(data)
    last_signal = data['signal'].iloc[-1]

    execute_trades(symbol, last_signal)


2023-11-25 11:27:42.787 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
